# Trabalho 2 - Cenário 3D

### Feito por:
* Caio Augusto Duarte Basso - NUSP 10801173
* Gabriel Garcia Lorencetti - NUSP 10691891

### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

### Inicializando janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
largura = 1280
altura = 720
window = glfw.create_window(largura, altura, "Trabalho 2 - Aldeia", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec3.
* Definindo matrizes Model, View e Projection que acumulam transformações geométricas 3D e permitem navegação no cenário.
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base nas transformações Model, View e Projection.

In [3]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        varying vec2 out_texture;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [4]:
fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [5]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [6]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [7]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [8]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [9]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [10]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront.


Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file


Nos slides e vídeo-aula da Aula 11 - Parte 1, nós descrevemos o funcionamento desse formato.

In [11]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])


        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat', 'o '):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model



In [12]:
glEnable(GL_TEXTURE_2D)
qtd_texturas = 30
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

### A lista abaixo armazena todos os vertices carregados dos arquivos

In [13]:
vertices_list = []    
textures_coord_list = []

### Vamos carregar cada modelo e definir funções para desenhá-los

In [14]:
def desenha_skybox():

    glEnable(GL_TEXTURE_2D)
    glDisable(GL_DEPTH_TEST)
    #glColor3f(1,1,1) --? 
    #face 1
    glBindTexture(GL_TEXTURE_2D, 16)
    glBegin(GL_QUADS)
    glTexCoord2f(0, 0)
    glVertex3f(-20.0, -1.0, -20.0)
    glTexCoord2f(1, 0)
    glVertex3f(20.0, -1.0, -20.0)
    glTexCoord2f(1, 1)
    glVertex3f(20.0, 19.0, -20.0)
    glTexCoord2f(0, 1)
    glVertex3f(-20.0, 19.0, -20.0)
    glEnd()
    #face 2
    glBindTexture(GL_TEXTURE_2D, 16)
    glBegin(GL_QUADS)
    glTexCoord2f(0, 0)
    glVertex3f(-20.0, -1.0, 20.0)
    glTexCoord2f(1, 0)
    glVertex3f(20.0, -1.0, 20.0)
    glTexCoord2f(1, 1)
    glVertex3f(20.0, 19.0, 20.0)
    glTexCoord2f(0, 1)
    glVertex3f(-20.0, 19.0, 20.0)
    glEnd()
    #face 3
    glBindTexture(GL_TEXTURE_2D, 16)
    glBegin(GL_QUADS)
    glTexCoord2f(0, 0)
    glVertex3f(20.0, -1.0, -20.0)
    glTexCoord2f(1, 0)
    glVertex3f(20.0, -1.0, 20.0)
    glTexCoord2f(1, 1)
    glVertex3f(20.0, 19.0, 20.0)
    glTexCoord2f(0, 1)
    glVertex3f(20.0, 19.0, -20.0)
    glEnd()
    #face 4
    glBindTexture(GL_TEXTURE_2D, 16)
    glBegin(GL_QUADS)
    glTexCoord2f(0, 0)
    glVertex3f(-20.0, -1.0, -20.0)
    glTexCoord2f(1, 0)
    glVertex3f(-20.0, -1.0, 20.0)
    glTexCoord2f(1, 1)
    glVertex3f(-20.0, 19.0, 20.0)
    glTexCoord2f(0, 1)
    glVertex3f(-20.0, 19.0, -20.0)
    glEnd()
    #face 5 top
    glBindTexture(GL_TEXTURE_2D, 16)
    glBegin(GL_QUADS)
    glTexCoord2f(0, 0)
    glVertex3f(20.0, 19.0, -20.0)
    glTexCoord2f(1, 0)
    glVertex3f(20.0, 19.0, 20.0)
    glTexCoord2f(1, 1)
    glVertex3f(-20.0, 19.0, 20.0)
    glTexCoord2f(0, 1)
    glVertex3f(-20.0, 19.0, -20.0)
    glEnd()
    #face 6 bottom
    glBindTexture(GL_TEXTURE_2D, 16)
    glBegin(GL_QUADS)
    glTexCoord2f(0, 0)
    glVertex3f(20.0, -1.0, -20.0)
    glTexCoord2f(1, 0)
    glVertex3f(20.0, -1.0, 20.0)
    glTexCoord2f(1, 1)
    glVertex3f(-20.0, -1.0, 20.0)
    glTexCoord2f(0, 1)
    glVertex3f(-20.0, -1.0, -20.0)
    glEnd()


    glEnable(GL_DEPTH_TEST)

#### Terreno

In [15]:
modelo = load_model_from_file('cenario/terreno/terreno.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo terreno.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo terreno.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(1,'cenario/terreno/grass.jpg')


Processando modelo terreno.obj. Vertice inicial: 0
Processando modelo terreno.obj. Vertice final: 6


#### Casa

In [16]:
modelo = load_model_from_file('cenario/aldeia/aldeia.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo aldeia.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo aldeia.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(2,'cenario/aldeia/aldeia.png')

Processando modelo aldeia.obj. Vertice inicial: 6
Processando modelo aldeia.obj. Vertice final: 2592


In [17]:
modelo = load_model_from_file('cenario/cama/cama.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cama.obj. Vertice inicial:',len(vertices_list))
faces_visited = []
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo cama.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(3,'cenario/cama/metal_tile.jpg')
load_texture_from_file(4,'cenario/cama/cloth.jpg')
load_texture_from_file(5,'cenario/cama/cloth_2.jpg')


Processando modelo cama.obj. Vertice inicial: 2592
Bad_Metal.001  vertice inicial = 2592
Cloth.001  vertice inicial = 6108
Pilow.001  vertice inicial = 6432
Processando modelo cama.obj. Vertice final: 6600


#### Fogão

In [18]:
modelo = load_model_from_file('cenario/fogao/fogao.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo fogao.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo fogao.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(6,'cenario/fogao/fogao.jpg')

Processando modelo fogao.obj. Vertice inicial: 6600
Processando modelo fogao.obj. Vertice final: 215709


#### Mesa

In [19]:
# modelo = load_model_from_file('cenario/mesa/mesa.obj')

# ### inserindo vertices do modelo no vetor de vertices
# print('Processando modelo mesa.obj. Vertice inicial:',len(vertices_list))
# faces_visited_mesa = []
# for face in modelo['faces']:
#     if face[2] not in faces_visited_mesa:
#         print(face[2], ' vertice inicial =', len(vertices_list))
#         faces_visited_mesa.append(face[2])
#     for vertice_id in face[0]:
#         vertices_list.append( modelo['vertices'][vertice_id-1] )
#     for texture_id in face[1]:
#         print(texture_id)
#         textures_coord_list.append( modelo['texture'][texture_id] )
# print('Processando modelo mesa.obj. Vertice final:',len(vertices_list))

# ### inserindo coordenadas de textura do modelo no vetor de texturas


# ### carregando textura equivalente e definindo um id (buffer): use um id por textura!
# load_texture_from_file(7,'cenario/mesa/madeira_vert.jpg')
# load_texture_from_file(8,'cenario/mesa/madeira_horiz.jpg')

#### Gato

In [20]:
modelo = load_model_from_file('cenario/gato/gato.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo gato.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo gato.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(7,'cenario/gato/gato.jpg')

Processando modelo gato.obj. Vertice inicial: 215709
Processando modelo gato.obj. Vertice final: 427437


#### Cacto

In [21]:
modelo = load_model_from_file('cenario/cacto/cacto.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cacto.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo cacto.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(8,'cenario/cacto/cacto.jpg')

Processando modelo cacto.obj. Vertice inicial: 427437
Processando modelo cacto.obj. Vertice final: 482445


#### Poço

In [22]:
modelo = load_model_from_file('cenario/poco/poco.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo poco.obj. Vertice inicial:',len(vertices_list))
faces_visited = []
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo poco.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(13,'cenario/poco/well_buket.tga')
load_texture_from_file(10,'cenario/poco/well_metal.tga')
load_texture_from_file(11,'cenario/poco/well_Roof.tga')
load_texture_from_file(14,'cenario/poco/Well_rope.tga')
load_texture_from_file(9,'cenario/poco/Well_Stone.tga')
load_texture_from_file(12,'cenario/poco/well_wood1.tga')


Processando modelo poco.obj. Vertice inicial: 482445
Material__36  vertice inicial = 482445
Material__26  vertice inicial = 482805
01___Default  vertice inicial = 484113
Material__40  vertice inicial = 484485
Material__38  vertice inicial = 484689
Material__41  vertice inicial = 487041
Processando modelo poco.obj. Vertice final: 518529


#### Mesa

In [23]:
modelo = load_model_from_file('cenario/mesa/mesa.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo mesa.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo mesa.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(15,'cenario/mesa/mesa.jpg')

Processando modelo mesa.obj. Vertice inicial: 518529
Processando modelo mesa.obj. Vertice final: 619713


In [24]:
load_texture_from_file(16,'cenario/ceu/ceu.jpg')

In [25]:
modelo = load_model_from_file('cenario/terreno/caminho.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo mesa.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo mesa.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(17,'cenario/terreno/pedra.jpg')

Processando modelo mesa.obj. Vertice inicial: 619713
Processando modelo mesa.obj. Vertice final: 619725


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar slots.

Nós agora vamos requisitar dois slots.
* Um para enviar coordenadas dos vértices.
* Outros para enviar coordenadas de texturas.

In [26]:
# Request a buffer slot from GPU
buffer = glGenBuffers(2)

###  Enviando coordenadas de vértices para a GPU

In [27]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)


###  Enviando coordenadas de textura para a GPU

In [28]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)





### Desenhando nossos modelos
* Cada modelo tem um Model para posicioná-los no mundo.
* É necessário saber qual a posição inicial e total de vértices de cada modelo
* É necessário indicar qual o ID da textura do modelo


In [29]:
def desenha_terreno():
    
    
    # aplica a matriz model
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.01; t_z = 0.0;
    
    # escala
    s_x = 20.0; s_y = 20.0; s_z = 20.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 1)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 0, 6) ## renderizando

In [30]:
def desenha_ceu():
    
    
    # aplica a matriz model
    
    # rotacao
    angle = 90.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.01; t_z = 0.0;
    
    # escala
    s_x = 20.0; s_y = 20.0; s_z = 20.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 1)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 0, 6) ## renderizando  


In [31]:
def desenha_casa():
    
    
    # aplica a matriz model
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.0; t_z = 0.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 2)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 6, 2628-6) ## renderizando

In [32]:
def desenha_cama():
    
    
    # aplica a matriz model
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = -5.0; t_y = -0.3; t_z = -0.5;
    
    # escala
    s_x = 1.3; s_y = 1.3; s_z = 1.3;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 3)    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 2592, 6108-2592) ## renderizando
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 4)    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 6108, 6432-6108) ## renderizando
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 5)    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 6432, 6600-6432) ## renderizando
    
    
# Processando modelo aldeia.obj. Vertice inicial: 2592
# Bad_Metal.001  vertice inicial = 2592
# Cloth.001  vertice inicial = 6108
# Pilow.001  vertice inicial = 6432
# Processando modelo aldeia.obj. Vertice final: 6600

In [33]:
def desenha_fogao():
    
    
    # aplica a matriz model
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.0; t_z = 0.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 6)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 6600, 215709-6600) ## renderizando

In [34]:
def desenha_gato(rotacao_inc):
    
    
    # aplica a matriz model
    
    # rotacao
    angle = rotacao_inc;
    r_x = 0.0; r_y = -1.0; r_z = 0.0;
    
    # translacao
    t_x = 0.0; t_y = -1.0; t_z = 0.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 7)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 215709, 427437-215709) ## renderizando


# Processando modelo gato.obj. Vertice inicial: 215709
# Processando modelo gato.obj. Vertice final: 427437

In [35]:
def desenha_cacto():
    
    
    # aplica a matriz model
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.0; t_z = 0.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 8)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 427437, 482445-427437) ## renderizando


# Processando modelo cacto.obj. Vertice inicial: 427437
# Processando modelo cacto.obj. Vertice final: 482445

In [36]:
def desenha_poco():
    
    
    # aplica a matriz model
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.0; t_z = 0.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 9)    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 482445, 482805-482445) ## renderizando
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 10)    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 482805, 484113-482805) ## renderizando
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 11)    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 484113, 484485-484113) ## renderizando
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 12)    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 484485, 484689-484485) ## renderizando
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 13)    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 484689, 487041-484689) ## renderizando
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 14)    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 487041, 518529-487041) ## renderizando


# Processando modelo poco.obj. Vertice inicial: 482445
# Material__36  vertice inicial = 482445
# Material__26  vertice inicial = 482805
# 01___Default  vertice inicial = 484113
# Material__40  vertice inicial = 484485
# Material__38  vertice inicial = 484689
# Material__41  vertice inicial = 487041
# Processando modelo poco.obj. Vertice final: 518529

In [37]:
def desenha_mesa():
    
    
    # aplica a matriz model
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.0; t_z = 0.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 15)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 518529, 619713-518529) ## renderizando

# Processando modelo mesa.obj. Vertice inicial: 518529
# Processando modelo mesa.obj. Vertice final: 619713

In [38]:
def desenha_caminho():
    
    
    # aplica a matriz model
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.00; t_z = 0.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 17)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 619713, 619725-619713) ## renderizando  
    
    
# Processando modelo mesa.obj. Vertice inicial: 619713
# Processando modelo mesa.obj. Vertice final: 619725

### Eventos para modificar a posição da câmera.

* Usei as teclas A, S, D e W para movimentação no espaço tridimensional
* Usei a posição do mouse para "direcionar" a câmera

In [39]:
cameraPos   = glm.vec3(0.0,  0.0,  1.0);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);
cameraTest  = glm.vec3(-1.0,  0.0,  0.0);


polygonal_mode = False

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode
    
    cameraSpeed = 0.2
    if cameraPos.x > 19: cameraPos.x = 19
    if cameraPos.x < -19: cameraPos.x = -19
    if cameraPos.y > 19: cameraPos.y = 19
    if cameraPos.y < -0.5: cameraPos.y = -0.5
    if cameraPos.z > 19: cameraPos.z = 19
    if cameraPos.z < -19: cameraPos.z = -19        
        
        
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed

    if key == 263 and (action==1 or action==2): # tecla A
        cameraPos += glm.normalize(glm.cross(cameraTest, cameraFront)) * cameraSpeed
        
    if key == 262 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraTest, cameraFront)) * cameraSpeed
        
    if key == 80 and action==1 and polygonal_mode==True:
        polygonal_mode=False
    else:
        if key == 80 and action==1 and polygonal_mode==False:
            polygonal_mode=True
    
        
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)


    
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)


### Matrizes Model, View e Projection

Teremos uma aula específica para entender o seu funcionamento.

In [40]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    
    angle = math.radians(angle)
    
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade

    
    # aplicando translacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))    
    
    # aplicando rotacao
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
    
    # aplicando escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform).T # pegando a transposta da matriz (glm trabalha com ela invertida)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Nesse momento, nós exibimos a janela!


In [41]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

In [42]:
glEnable(GL_DEPTH_TEST) ### importante para 3D

rotacao_inc = 0
while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
    
    rotacao_inc += 0.7
    
    desenha_skybox()
    desenha_terreno()
    desenha_caminho()
    desenha_casa()
    desenha_cama()
    desenha_fogao()
    desenha_gato(rotacao_inc)
    desenha_cacto()
    desenha_poco()
    desenha_mesa()
    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_FALSE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_FALSE, mat_projection)
    
    glfw.swap_buffers(window)

glfw.terminate()